# 单量子比特门分解算法

## 1 课题背景：

在容错量子计算中，减少量子过程产生误差的一种常用方法是采用高精度基本门集合来实现任意门操作。典型的基本门集合包括H门、T门、CNOT门。理论研究表明，任何单量子比特门，给定一个特定的精度，总是可以表示为一个有限长度的$H,T$门序列。反之，可以通过为序列设置一个固定的长度来逼近目标量子门，随着序列长度的增加，逼近精度提高。
理论上，蛮力搜索总是可以找到任何单量子位门分解的最优解（计算搜索空间中每个可能的门序列与目标门的距离），但其计算复杂度随着序列长度的增加呈指数增长，在实际应用中难以实现。为了应对这一挑战，研究者们已经提出了多种方法，包括SK算法，中间相遇算法，KD树算法等。

## 2 项目任务及实现方案：

本项目的核心任务是实现一种使用启发式算法高效分解任意的单量子比特门的方法，展现出整体求解时间或者求解返回的基础量子门数量相比之前的经典算法具有不小于一个数量级以上的优势。

### SK算法

本文代码实现了近似单比特量子门的 Solovay-Kitaev 算法（SK 算法），其目标是将任意单比特 $SU(2)$ 单位门，逼近为基本门集合（$H$ 和 $T$ 门）的组合。

1.导入相关依赖

In [1]:
import random
from random import randint, seed
import numpy as np
import scipy as sp
from src import helper
from src import ops
from src import state
import math
import time

ops、helper、state：辅助模块，定义了量子门、状态和辅助函数。

2.工具函数定义

In [2]:
def to_su2(u):

  det = u[0][0] * u[1][1] - u[0][1] * u[1][0]
  return np.sqrt(1 / det) * u

def F_dist(u, v):

  return math.sqrt(1 - 0.5 * np.trace(u.adjoint() @ v))

def generate_su2(s):
  seed(s)
  theta =[randint(1,10) for i in range(3)]
  t =np.linalg.norm(theta)
  theta[0] = theta[0]/t
  theta[1] = theta[1]/t
  theta[2] = theta[2]/t
  su2 =  ops.Rotation(theta,t,'s')
  return su2

def create_unitaries(base, limit):

  gate_list = []
  for width in range(limit):
    for bits in helper.bitprod(width):
      u = ops.Identity()
      for bit in bits:
        u = u @ base[bit]
      gate_list.append(u)
  return gate_list

def create_unitaries_mitm(base, width):

  gate_list = []
  for bits in helper.bitprod(width):
    u = ops.Identity()
    for bit in bits:
      u = u @ base[bit]
    gate_list.append(u)
  return gate_list


to_su2将任意 2x2 单位矩阵转化为 $SU(2)$ 矩阵

F_dist定义了矩阵之间的距离度量

generate_su2用于生成随机的$SU(2)$ 矩阵

create_unitaries用于生成所有 0-limit 位的 bit 串，如：0, 1, 00, 01, 10, ... ，用来构造SK搜索空间。 create_unitaries_mitm适用于中间相遇策略。

4.初始估计方法

SK算法作为一种递推算法，需要以一个初始估计作为递推的起点，下面是两种初始估计方法：暴力搜索和中间相遇方法

In [3]:
def bruce(gate_list, u):

  min_dist, min_u = 10, ops.Identity()
  for gate in gate_list:
    tr_dist = F_dist(gate, u)
    if tr_dist < min_dist:
      min_dist, min_u = tr_dist, gate
  return min_u

def mitm(G,u):
  
  min_dist, min_u = 10, ops.Identity()
  for i in range(len(G)-1):
    for w in G[i+1]:
      for v in G[i]:
        a=to_su2(np.dot(v,w))
        tr_dist = F_dist(a, u)
        if tr_dist < min_dist:
           min_dist, min_u = tr_dist, a
      for v in G[i+1]:
        a=to_su2(np.dot(v,w))
        tr_dist = F_dist(a, u)
        if tr_dist < min_dist:
           min_dist, min_u = tr_dist, a
  return min_u

暴力搜索直接线性搜索gate_list中的最接近项，而中间相遇（Meet-in-the-Middle）则是通过双向搜索逼近加速查找。

5.GC分解

SK算法的递归关键在于使用了一个所谓的GC分解的技术，通过两个操作的来回抵消，将误差压缩至更低阶。

In [4]:
def u_to_bloch(u):

  angle = np.real(np.arccos((u[0, 0] + u[1, 1]) / 2))
  sin = np.sin(angle)
  if sin < 1e-10:
    axis = [0, 0, 1]
  else:
    nx = (u[0, 1] + u[1, 0]) / (2j * sin)
    ny = (u[0, 1] - u[1, 0]) / (2 * sin)
    nz = (u[0, 0] - u[1, 1]) / (2j * sin)
    axis = [nx, ny, nz]
  return axis, 2 * angle


def gc_decomp(u):

  def diagonalize(u):
    _, v = np.linalg.eig(u)
    return ops.Operator(v)

  axis, theta = u_to_bloch(u)

  phi = 2.0 * np.arcsin(np.sqrt(np.sqrt((0.5 - 0.5 * np.cos(theta / 2)))))

  v = ops.RotationX(phi)
  if axis[2] > 0:
    w = ops.RotationY(2 * np.pi - phi)
  else:
    w = ops.RotationY(phi)

  ud = diagonalize(u)
  vwvdwd = diagonalize(v @ w @ v.adjoint() @ w.adjoint())
  s = ud @ vwvdwd.adjoint()

  v_hat = s @ v @ s.adjoint()
  w_hat = s @ w @ s.adjoint()
  return v_hat, w_hat

6. Solovay-Kitaev 主递归算法

In [5]:
def sk_algo(u, gates, n, est):
  """Solovay-Kitaev Algorithm."""

  if n == 0:
    if est == 'mitm':
      return mitm(gates, u)
    if est == 'bruce':
      return bruce(gates, u)
  else:
    u_next = sk_algo(u, gates, n - 1, est)
    v, w = gc_decomp(u @ u_next.adjoint())
    v_next = sk_algo(v, gates, n - 1, est)
    w_next = sk_algo(w, gates, n - 1, est)
    return v_next @ w_next @ v_next.adjoint() @ w_next.adjoint() @ u_next

当 n=0 时，调用初始估计的搜索策略（bruce/mitm）

否则：通过GC分解递归逼近误差项的 commutator，然后组合还原目标门。

7.主程序

In [6]:
def main():
  
  num_experiments = 1           #随机生成SU2的个数
  depth = 6                     #基本门序列长度
  recursion = 3                 #SK算法深度
  est='bruce'                   #初始估计方法

  print('SK algorithm - est: {}, depth: {}, recursion: {}, experiments: {}'.
        format(est, depth, recursion, num_experiments))
  
  base = [to_su2(ops.Hadamard()), to_su2(ops.Tgate())]

  t1=time.time()

  if est=='mitm':
    depth = math.ceil(depth/2)+1
    G = [create_unitaries_mitm(base, i) for i in range(depth)]
  if est=='bruce':
    G = create_unitaries(base, depth+1)

  sum_dist = 0.0
  for s in range(num_experiments):
    u = generate_su2(s)
    u_approx = sk_algo(u, G, recursion, est)
    dist = F_dist(u, u_approx)
    dist= math.log(dist,10)
    sum_dist += dist
    print('Distance: {:.6f}'. format(dist))

  t2=time.time()
  t=t2-t1
  print('Mean time:{:.4f},  Mean Dist: {:.6f}'.
        format(t/num_experiments, sum_dist / num_experiments))

其中，num_experiments为随机生成SU2的个数，depth为基本门序列长度，recursion为SK算法深度，est为初始估计方法。

In [7]:
if __name__ == '__main__':
  main()

SK algorithm - est: bruce, depth: 6, recursion: 3, experiments: 1
Distance: -1.464093
Mean time:0.1329,  Mean Dist: -1.464093


C:\Users\97849\AppData\Local\Temp\ipykernel_17716\3549546427.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.sqrt(1 - 0.5 * np.trace(u.adjoint() @ v))


运行main函数，打印出SK算法得到的门序列与目标门之间的距离（取log）以及算法运行的时间。

## 3 参考文献
[1] Dawson C M, Nielsen M A. The solovay-kitaev algorithm[J]. arXiv preprint quant-ph/0505030, 2005. https://arxiv.org/pdf/quant-ph/0505030

[2] Amy M, Maslov D, Mosca M, et al. A meet-in-the-middle algorithm for fast synthesis of depth-optimal quantum circuits[J]. IEEE Transactions on Computer-Aided Design of Integrated Circuits and Systems, 2013, 32(6): 818-830. https://arxiv.org/pdf/1206.0758